In [1]:
import graphviz
import numpy as np
import Frozen_Lake as fl

env = fl.FrozenLakeEnv(map_name="4x4")

env.reset()

In [5]:
env.get_all_states()

((0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3))

In [66]:
def init_policy(env):

    actions_set = set()

    row_max = 0
    col_max = 0
    for state in env.get_all_states():
        actions_set.update(env.get_possible_actions(state))
        row_max = max(row_max, state[0])
        col_max = max(col_max, state[1])

    actions = {i:action for i,action in enumerate(sorted(actions_set))}

    policy = np.zeros((row_max+1, col_max+1, len(actions_set)))

    print('policies    |states')
    for y,x in env.get_all_states():
        possible_actions = env.get_possible_actions((y,x))
        if len(possible_actions)!=0:
            uniform_prob = 1/len(possible_actions)
            policy[y][x] = uniform_prob
        print(policy[y][x], y, x,)
    return policy, list(sorted(actions_set))

In [67]:
policy,actions_set = init_policy(env)

policies    |states
[0.25 0.25 0.25 0.25] 0 0
[0.25 0.25 0.25 0.25] 0 1
[0.25 0.25 0.25 0.25] 0 2
[0.25 0.25 0.25 0.25] 0 3
[0.25 0.25 0.25 0.25] 1 0
[0. 0. 0. 0.] 1 1
[0.25 0.25 0.25 0.25] 1 2
[0. 0. 0. 0.] 1 3
[0.25 0.25 0.25 0.25] 2 0
[0.25 0.25 0.25 0.25] 2 1
[0.25 0.25 0.25 0.25] 2 2
[0. 0. 0. 0.] 2 3
[0. 0. 0. 0.] 3 0
[0.25 0.25 0.25 0.25] 3 1
[0.25 0.25 0.25 0.25] 3 2
[0. 0. 0. 0.] 3 3


In [68]:
policy, actions_set

(array([[[0.25, 0.25, 0.25, 0.25],
         [0.25, 0.25, 0.25, 0.25],
         [0.25, 0.25, 0.25, 0.25],
         [0.25, 0.25, 0.25, 0.25]],
 
        [[0.25, 0.25, 0.25, 0.25],
         [0.  , 0.  , 0.  , 0.  ],
         [0.25, 0.25, 0.25, 0.25],
         [0.  , 0.  , 0.  , 0.  ]],
 
        [[0.25, 0.25, 0.25, 0.25],
         [0.25, 0.25, 0.25, 0.25],
         [0.25, 0.25, 0.25, 0.25],
         [0.  , 0.  , 0.  , 0.  ]],
 
        [[0.  , 0.  , 0.  , 0.  ],
         [0.25, 0.25, 0.25, 0.25],
         [0.25, 0.25, 0.25, 0.25],
         [0.  , 0.  , 0.  , 0.  ]]]),
 ['down', 'left', 'right', 'up'])

Policy Evaluation

Value Function
$$v_\pi(s)=\mathbb{E}_\pi\left[G\right]$$


$$v^{k+1}(s)=\sum_{a}\pi(a|s)\left(\mathcal{R}(s,a)+\gamma\sum_{s'}\mathcal{P}(s'|s,a)v^k(s')\right), s\in{S}$$
or
$$v^{k+1}=\mathcal{R}_\pi+\gamma\mathcal{P}_{\pi}v^k$$

Value Function in deterministic Case: 
$$v_\pi(s)=G(\tau_\pi)$$

Action-Value Function

$$q_{\pi}(s,a)=\mathbb{E}_\pi\left[G|S_0=s,A_0=a\right]$$


$$q_{\pi}(s,a)=\mathcal{R}(s_0,a_0)+\mathcal{R}(s,a)+\gamma\sum_{s'}\mathcal{P}(s'|s,a)v_{\pi}(s')$$

$$q_{\pi}(s,a)=\mathcal{R}(s,a)+\gamma\sum_{s'}\left(\mathcal{P}(s'|s,a)\sum_{a'}\pi(a'|s')q(_{\pi}(s',a')\right)$$







### Algorithm:
Let $\pi^0$ - это начальный полиси and $L,K\in\mathbb{N}$. $%invisible$
$%https://www.overleaf.com/learn/latex/Mathematical_fonts$
For each $k \in \overline{0,K}$,do

- (Policy Evaluation) Iterative Policy Evaluation
$v^{l+1}=R_{\pi^k}+P_{\pi^k}v^l, l\in\overline{0,L-1}$

Define $q^L(s,a)$ by $v^L(s)$

- (Policy improvement) Greedy Policy Imporvement

$$\pi^{k+1}(a|s) = \begin{cases}
1, \text{if } a \in \text{argmax}_{a'\in\mathbb{A}}q^L(s,a')\\
0, \text{otherwise}
\end{cases}$$



In [10]:
actions_dict ={action:i for i,action in enumerate(sorted(actions_set))}
actions_dict

{'down': 0, 'left': 1, 'right': 2, 'up': 3}

In [12]:
env.render()

*FFF
FHFH
FFFH
HFFG



In [13]:
gamma = 0.9


In [15]:
def init_values(policy):
     return np.zeros((policy.shape[0]*policy.shape[1],))
values = init_values(policy)
values

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Initial values:
$$v^{0}=\mathcal{R}_{\pi^k}$$


$$v^{l+1}=\mathcal{R}_{\pi^k}+\mathcal{P}_{\pi^k}v^l$$
$$v^{l+2}=\mathcal{R}_{\pi^k}+\mathcal{P}_{\pi^k}v^{l+1}$$
$$v^{l+2}=\mathcal{R}_{\pi^k}+\mathcal{P}_{\pi^k}(\mathcal{R}_{\pi^k}+\mathcal{P}_{\pi^k}v^l)$$

$$v^{k+1}(s)=\sum_{a}\pi(a|s)\left(\mathcal{R}(s,a)+\gamma\sum_{s'}\mathcal{P}(s'|s,a)v^k(s')\right), s\in{S}$$

NO: $$\mathcal{R}(s,a)=\sum_{s'}\pi(s'|a,s)\mathcal{R}(s,a,s')$$ - all possible transitions, mult by probs and sum of it

$$v^{k+1}(s)=\sum_{a}\pi(a|s)\left(\sum_{s'}\mathcal{R}(s,a,s')+\gamma\sum_{s'}\mathcal{P}(s'|s,a)v^k(s')\right), s\in{S}$$

$$v^{k+1}(s)=\sum_{a}\left(\pi(a|s)\sum_{s'}\mathcal{R}(s,a,s')+\gamma\pi(a|s)\sum_{s'}\mathcal{P}(s'|s,a)v^k(s')\right), s\in{S}$$

$$v^{k+1}(s)=\sum_{a}\left(\sum_{s'}\pi(a|s)\mathcal{R}(s,a,s')+\gamma\sum_{s'}\pi(a|s)\mathcal{P}(s'|s,a)v^k(s')\right), s\in{S}$$

$$v^{k+1}(s)=\sum_{a}\left(\sum_{s'}\left(\pi(a|s)\mathcal{R}(s,a,s')+\gamma\pi(a|s)\mathcal{P}(s'|s,a)v^k(s')\right)\right), s\in{S}$$

$$v^{k+1}(s)=\sum_{a}\left(\sum_{s'}\pi(a|s)\left(\mathcal{R}(s,a,s')+\gamma\mathcal{P}(s'|s,a)v^k(s')\right)\right), s\in{S}$$

In [16]:
for state in env.get_all_states():
    for action in env.get_possible_actions(state):
        for next_state in env.get_next_states(state, action):
            prob = env.get_transition_prob(state, action, next_state)
            reward = env.get_reward(state, action, next_state)
            if reward!=0:
                print(reward,state, action, next_state, prob)

1.0 (3, 2) down (3, 3) 0.1
1.0 (3, 2) right (3, 3) 0.8
1.0 (3, 2) up (3, 3) 0.1


In [17]:
env.render()

*FFF
FHFH
FFFH
HFFG



In [18]:
def policy_evaluation_step(policy, values, gamma):
    new_values = np.zeros(policy.shape[0]*policy.shape[1])

    # policy evaluation

    for state in env.get_all_states():
        state_y, state_x = state
        idx_new_values = state_y*policy.shape[0] + state_x
        for action in env.get_possible_actions(state):
            policy_prob = policy[state_y][state_x][actions_dict[action]]
            mean_reward =0

            total_reward = 0
            total_value = 0
            for next_state in env.get_next_states(state, action):
                next_state_y, next_state_x = next_state
                idx_old_values = next_state_y*policy.shape[0] + next_state_x
                # reward
                reward = env.get_reward(state, action, next_state)
                total_reward += reward
                # value
                trans_prob = env.get_transition_prob(state, action, next_state)
                value_func = values[idx_old_values]
                total_value += gamma * trans_prob * value_func

            new_values[idx_new_values] += \
                policy_prob * (total_reward + total_value)

    return new_values

In [19]:
values = policy_evaluation_step(policy, values, gamma)
values

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.75, 0.  ])

$$q_{\pi}(s,a)=\mathcal{R}(s,a)+\gamma\sum_{s'}\mathcal{P}(s'|s,a)v_\pi(s')$$
since we have specific environment, that give you reward for transitioning to specific satet
$$q_{\pi}(s,a)=\sum_{s'}\mathcal{R}(s',s,a)+\gamma\sum_{s'}\mathcal{P}(s'|s,a)v_\pi(s')$$

$$q_{\pi}(s,a)=\sum_{s'}\left(\mathcal{R}(s',s,a)+\gamma\mathcal{P}(s'|s,a)v_\pi(s')\right)$$

In [20]:
# как опеределить дурацкий Ку
# это матрица состояний на действия со значение награды.

q = np.zeros(policy.shape) #(S,A)

In [93]:
actions_dict

{'down': 0, 'left': 1, 'right': 2, 'up': 3}

In [22]:
def Q(policy, q):
    new_q = np.zeros(policy.shape)

    for state in env.get_all_states():
        state_y, state_x = state
        # actions_dict[action]
        for action in env.get_possible_actions(state):
            for next_state in env.get_next_states(state, action):
                next_state_y, next_state_x = state

                reward = env.get_reward(state, action, next_state)
                prob = env.get_transition_prob(state, action, next_state)
                value = values[next_state_y*policy.shape[0]+next_state_x]

                new_q[state_y][state_x][actions_dict[action]] \
                    = reward + gamma*prob*value
    return new_q

- (Policy improvement) Greedy Policy Imporvement
$$\pi^{k+1}(a|s) = \begin{cases}
1, \text{if } a \in \text{argmax}_{a'\in\mathbb{A}}q^L(s,a')\\
0, \text{otherwise}
\end{cases}$$


In [23]:
L = 100
K = 100

In [25]:
values = init_values(policy)
for k in range(K):
    values = policy_evaluation_step(policy, values, gamma)
values

array([0.01343178, 0.01266737, 0.03020027, 0.01235466, 0.02016588,
       0.        , 0.07900113, 0.        , 0.05602845, 0.17282102,
       0.32091584, 0.        , 0.        , 0.39114915, 1.17447048,
       0.        ])

In [34]:
env.get_next_states((3,2), 'left')

/home/shmalex/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{(2, 2): 0.1, (3, 1): 0.8, (3, 2): 0.1}

In [35]:
env.get_next_states((3,2), 'right')

{(3, 2): 0.1, (3, 3): 0.8, (2, 2): 0.1}

In [26]:
q = Q(policy, q)
q

array([[[0.00120886, 0.00120886, 0.00120886, 0.01087974],
        [0.00114006, 0.00114006, 0.00114006, 0.00114006],
        [0.00271802, 0.00271802, 0.00271802, 0.00271802],
        [0.00111192, 0.00111192, 0.01000727, 0.00111192]],

       [[0.00181493, 0.00181493, 0.00181493, 0.00181493],
        [0.        , 0.        , 0.        , 0.        ],
        [0.0071101 , 0.0071101 , 0.0071101 , 0.0071101 ],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.00504256, 0.00504256, 0.00504256, 0.00504256],
        [0.01555389, 0.01555389, 0.01555389, 0.01555389],
        [0.02888243, 0.02888243, 0.02888243, 0.02888243],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        , 0.        ],
        [0.03520342, 0.03520342, 0.03520342, 0.03520342],
        [1.10570234, 0.10570234, 0.10570234, 0.10570234],
        [0.        , 0.        , 0.        , 0.        ]]])

In [27]:
actions_set

['down', 'left', 'right', 'up']

In [84]:
def policy_imporvement(policy, q):
    next_policy = np.zeros(policy.shape)
    for state in env.get_all_states():
        state_y,state_x = state

        idx_action = np.argmax(q[state_y][state_x])
        next_policy[state_y][state_x][idx_action] = 1
    return next_policy

In [29]:
actions_dict

{'down': 0, 'left': 1, 'right': 2, 'up': 3}

In [31]:
L = 100
K = 100
gamma = 0.9

In [85]:
policy, actions = init_policy(env)
values = init_values(policy)

policies    |states
[0.25 0.25 0.25 0.25] 0 0
[0.25 0.25 0.25 0.25] 0 1
[0.25 0.25 0.25 0.25] 0 2
[0.25 0.25 0.25 0.25] 0 3
[0.25 0.25 0.25 0.25] 1 0
[0. 0. 0. 0.] 1 1
[0.25 0.25 0.25 0.25] 1 2
[0. 0. 0. 0.] 1 3
[0.25 0.25 0.25 0.25] 2 0
[0.25 0.25 0.25 0.25] 2 1
[0.25 0.25 0.25 0.25] 2 2
[0. 0. 0. 0.] 2 3
[0. 0. 0. 0.] 3 0
[0.25 0.25 0.25 0.25] 3 1
[0.25 0.25 0.25 0.25] 3 2
[0. 0. 0. 0.] 3 3


In [86]:
# Policy Evaluation

In [87]:
values = init_values(policy)
for k in range(K):
    values = policy_evaluation_step(policy, values, gamma)
    q = np.zeros(policy.shape)
    q = Q(policy, q)
    policy = policy_imporvement(policy, q)
policy

array([[[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]])

In [69]:
from collections import Counter

In [70]:
actions_counter = Counter()
for state in env.get_all_states():
    actions = env.get_possible_actions(state)
    actions_counter.update(['-'.join(actions)])

In [71]:
actions_counter

Counter({'left-down-right-up': 11, '': 5})

In [59]:
policy

/home/shmalex/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ]]])

In [94]:
total_reward = 0
state = env.reset()
for _ in range(100):
    possible_actions = list(sorted(env.get_possible_actions(state)))
    print(possible_actions)
    state_y, state_x = state
    actions_distr = policy[state_y][state_x]
    
    action = np.random.choice(possible_actions,p=actions_distr)
    state, reward, done, _ = env.step(action)
    print(state, reward, done, _,action, actions_distr)
    env.render()
    total_reward += reward
    
    if done: break

['down', 'left', 'right', 'up']
(0, 0) 0.0 False {} up [0. 0. 0. 1.]
*FFF
FHFH
FFFH
HFFG

['down', 'left', 'right', 'up']
(0, 0) 0.0 False {} up [0. 0. 0. 1.]
*FFF
FHFH
FFFH
HFFG

['down', 'left', 'right', 'up']
(0, 0) 0.0 False {} up [0. 0. 0. 1.]
*FFF
FHFH
FFFH
HFFG

['down', 'left', 'right', 'up']
(0, 0) 0.0 False {} up [0. 0. 0. 1.]
*FFF
FHFH
FFFH
HFFG

['down', 'left', 'right', 'up']
(0, 1) 0.0 False {} up [0. 0. 0. 1.]
S*FF
FHFH
FFFH
HFFG

['down', 'left', 'right', 'up']
(1, 1) 0.0 True {} down [1. 0. 0. 0.]
SFFF
F*FH
FFFH
HFFG



In [48]:
new_state

(1, 1)